In [7]:
import pandas as pd
import numpy as np
import re

import nltk
nltk.download('omw-1.4')

## for w2v
import gensim
import gensim.downloader as gensim_api
## for bert
import transformers
from sklearn import metrics, manifold


# df=pd.read_csv("/Users/shravankaul/Desktop/UWM/STAT 628 project/yelp_dataset_2022/data_cleaned.csv")
df=pd.read_csv("/Users/wangjiayang/Desktop/STAT628/yelp_dataset_2022/data_cleaned_PA.csv")

res=df
print (res.groupby('state')['business_id'].nunique())

options = ['PA'] 
res = res[res['state'].isin(options)] 
res= res[['user_id','business_id','review_id','text_x','stars_y']]

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/wangjiayang/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


state
PA    1122
Name: business_id, dtype: int64


In [8]:
len(res)

87904

In [9]:
res.reset_index(drop=True, inplace=True)

In [10]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [11]:
def embed(tokens, nlp):

    lexemes = (nlp.vocab[token] for token in tokens)

    vectors = np.asarray([
        lexeme.vector
        for lexeme in lexemes
        if lexeme.has_vector
        and not lexeme.is_stop
        and len(lexeme.text) > 1
    ])

    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
    else:
        width = nlp.meta['vectors']['width']  # typically 300
        centroid = np.zeros(width)

    return centroid

In [13]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text


lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords.append('fuck')
lst_stopwords.append('dick')
lst_stopwords.append('prick')

res["text_clean"] = res["text_x"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))

docs= res["text_clean"]


In [14]:

label_names = ['food', 'drink', 'price', 'service', 'ambience']

In [15]:
doc=docs[0]
doc

'place top notch phenomenal service fantastic ambiance go elsewhere food looked smelled amazing love creative everything cant stop daydreaming food hand best brunch experience life dont even like french toast usually there far delicious food thats ever entered mouth literally drooling writing chicken waffle unreal standout love flinstones pushuppop drink creative clever even deviled egg best ive personally ever everything right even come music super fun great experience ignore review thats 4 star must delusional'

In [16]:
tokens = doc.split(' ')
centroid = embed(tokens, nlp)
centroid.shape

(300,)

In [17]:
label_vectors = np.asarray([
     embed(label.split(' '), nlp)
    for label in label_names
 ])


In [18]:
from sklearn.neighbors import NearestNeighbors
from scipy import spatial
neigh = NearestNeighbors(n_neighbors=5,metric=spatial.distance.cosine)
neigh.fit(label_vectors)

closest_label = neigh.kneighbors([centroid], return_distance=True)
print(closest_label)


(array([[0.38465881, 0.41406149, 0.5038673 , 0.62769869, 0.7351242 ]]), array([[4, 0, 1, 3, 2]]))


In [19]:

gg={closest_label[1][0][i]: closest_label[0][0][i] for i in range(len(closest_label[1][0]))}
gg

{4: 0.3846588134765625,
 0: 0.4140614867210388,
 1: 0.5038672983646393,
 3: 0.627698689699173,
 2: 0.7351242005825043}

In [20]:
def predict(doc, nlp, neigh):
    tokens = doc.split(' ')
    centroid = embed(tokens, nlp)
    closest_label = neigh.kneighbors([centroid], return_distance=True)
    return closest_label[0][0],closest_label[1][0]

dist,label = map(list, zip(*[predict(doc, nlp, neigh)  for doc in docs]))


/Users/wangjiayang/opt/anaconda3/lib/python3.8/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/Users/wangjiayang/opt/anaconda3/lib/python3.8/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/Users/wangjiayang/opt/anaconda3/lib/python3.8/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/Users/wangjiayang/opt/anaconda3/lib/python3.8/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/Users/wangjiayang/opt/anaconda3/lib/python3.8/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [21]:
zero_data = np.zeros(shape=(len(label),len(label[0])))
df_classify = pd.DataFrame(zero_data,columns=["0","1","2","3","4"])

for i in range(len(label)):
    for j in range(len(label[0])):
        if label[i][j]==0:
            df_classify["0"][i]=dist[i][j]
        elif label[i][j]==1:
            df_classify["1"][i]=dist[i][j]
        elif label[i][j]==2:
            df_classify["2"][i]=dist[i][j]
        elif label[i][j]==3:
            df_classify["3"][i]=dist[i][j]
        elif label[i][j]==4:
            df_classify["4"][i]=dist[i][j]
     

In [22]:
df_classify.columns=['food', 'drink', 'price', 'service', 'ambience']

In [23]:

df_classify[:2]

,food,drink,price,service,ambience
0,0.414061,0.503867,0.735124,0.627699,0.384659
1,0.511236,0.477190,0.722486,0.676104,0.447329


In [24]:
df_classify.loc[df_classify['food'] > 0.70, 'food'] = 1
df_classify.loc[df_classify['drink'] > 0.70, 'drink'] = 1
df_classify.loc[df_classify['price'] > 0.70, 'price'] = 1
df_classify.loc[df_classify['service'] > 0.70, 'service'] = 1
df_classify.loc[df_classify['ambience'] > 0.70, 'ambience'] = 1

In [25]:
df_classify = 1-df_classify
df_classify

,food,drink,price,service,ambience
0,0.585939,0.496133,0.000000,0.372301,0.615341
1,0.488764,0.522810,0.000000,0.323896,0.552671
2,0.609678,0.638099,0.000000,0.000000,0.407781
3,0.579838,0.522945,0.496539,0.428235,0.541877
4,0.638008,0.558445,0.407505,0.381120,0.509049
...,...,...,...,...,...
87899,0.645746,0.611957,0.300618,0.000000,0.445449
87900,0.580656,0.561727,0.372583,0.430459,0.531810
87901,0.483534,0.586383,0.000000,0.000000,0.418747
87902,0.532742,0.530287,0.395059,0.335536,0.557534


In [26]:
#Sentiment Analysis
gg=res[:10]
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
def flair_prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    sentence.labels[0].to_dict()
    attitude=sentence.labels[0].value
    score=sentence.labels[0].score
    if attitude=='NEGATIVE':
        score=-score
    return score
res["sentiment score"] = res["text_clean"].apply(flair_prediction)






2022-12-07 17:46:51,613 loading file /Users/wangjiayang/.flair/models/sentiment-en-mix-distillbert_4.pt


In [27]:
res[:10]

,user_id,business_id,review_id,text_x,stars_y,text_clean,sentiment score
0,dldfg-X_QbBkhR2DOsQFWg,QdN72BWoyFypdGJhhI5r7g,z_fgvINjKZCw5RgByaTxxw,"This place is top notch, with phenomenal servi...",5,place top notch phenomenal service fantastic a...,0.988117
1,3Nk8hO7cvu0GTJxmKyCOBw,QdN72BWoyFypdGJhhI5r7g,O0y-JZOppUbRTmU8IIXazw,Came here on a Saturday afternoon for brunch w...,5,came saturday afternoon brunch friend let say ...,0.994090
2,yJw-NqN5gW1wMGfSSMqBMw,QdN72BWoyFypdGJhhI5r7g,-O8dohaJ4nI_e_64jgsxLw,This place is INCREDIBLE! It's American food w...,5,place incredible american food twist start dri...,0.994347
3,3IFfnwuFB-Y_0vSOlqNpSQ,QdN72BWoyFypdGJhhI5r7g,jstzd6bBAiEhyB2b7cy0dg,Outrageously overpriced for what it is. Insult...,2,outrageously overpriced insultingly would one ...,-0.999409
4,IJTd6GhZd4hEoW8l4VoKEA,QdN72BWoyFypdGJhhI5r7g,uoMpzCdQ4-KYANZQWQudtg,"Ekks, how hard is it to serve food that isn't ...",1,ekks hard serve food isnt raw hate giving 1 st...,-0.999929
5,nBUnQnecIYYWuxk0g5njhw,QdN72BWoyFypdGJhhI5r7g,DiC22Mh6D_a3k5UhDjtEzg,Food here is AWESOME. Buffalo chicken egg roll...,5,food awesome buffalo chicken egg roll lamb bur...,0.996307
6,W4K4WEmzNVqbyF9oySrwBg,QdN72BWoyFypdGJhhI5r7g,Ygd_24TJuxeOJc4EEZJw6A,The best new bar in Philly. Amazing drink menu...,5,best new bar philly amazing drink menu taste g...,0.988230
7,YoIDy9T9_9UGljau5mKxcA,QdN72BWoyFypdGJhhI5r7g,wVKYcJNyS0u8I3VYebERbw,We are obsessed with this place. The bartender...,5,obsessed place bartender friendly drink taste ...,0.995959
8,vi6mqCFZitRwo_I85Osg5g,QdN72BWoyFypdGJhhI5r7g,A78ZQcwZnSLE_yMcXzyPGQ,Have been here twice with my boyfriend. We had...,4,twice boyfriend great food service time menu g...,-0.578572
9,6bxzJaXxGCqApmJsyIlr9w,QdN72BWoyFypdGJhhI5r7g,UjkWheIk6XFzHAVNZL8D_g,Stop waiting to come here and just eat a meal ...,4,stop waiting come eat meal already food simple...,0.519640


NameError: name 'df_classify' is not defined

In [29]:
df_comb = pd.concat([res, df_classify], axis=1)

In [31]:
res.rename(columns = {'sentiment score':'sentiment_score'}, inplace = True)
result = df_classify[['food', 'drink', 'price', 'service', 'ambience']].multiply(res["sentiment_score"], axis="index")

df_comb = pd.concat([res, result], axis=1)

final_grouped=df_comb.groupby("business_id").mean()
final_grouped.to_csv("final_sent_grouped_business_id_closed.csv")